In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.part_number/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part={input_.loc[a, 'Part Number']}&func=PART&vid='''
                        check_url = input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            dict_dict = resp.json()
                            if dict_dict:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                dict_attr, dict_ = {}, dict_dict['pd']
                for key, value in dict_.items():
                    if key not in ['rowId', 'basePart', 'maxRows'] and value != None:
                        if type(value) in [int, float]:
                            dict_attr[key.strip().title()] = value
                        elif type(value) == str:
                            dict_attr[key.strip().title()] = value.strip()
                        else:
                            raise
            
                # = = = = = = = = = = = = = = =

                vehicle = '\n'.join([dict_vehicle['buyersGuideDesc'].strip() for dict_vehicle in dict_dict['pp']['buyersGuides']])

                # = = = = = = = = = = = = = = =

                list_spec, dict_count = dict_dict['pp']['partSpecs'], {}
                for i, spec in enumerate(list_spec):
                    key = spec['attributeName_en'].strip()
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                
                dict_spec, dict_no = {}, {}
                for i, spec in enumerate(list_spec):
                    key = spec['attributeName_en'].strip()
                    if dict_count[key] == 1:
                        dict_spec[str(i)] = {key: spec['siteValue_en'].strip()}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_spec[str(i)] = {f'''{key}-{dict_no[key]}''': spec['siteValue_en'].strip()}
        
                # = = = = = = = = = = = = = = =

                service = '\n'.join([f'''{i+1}. {service['serviceName'].strip()}''' for i, service in enumerate(dict_dict['pp']['completeRepair']['serviceSpec'])])

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Vehicle'] = vehicle
                df_temp.loc[0, 'Service'] = service
                df_temp = pd.concat([df_temp, pd.DataFrame([dict_attr])], axis=1).fillna('')
                df_temp.loc[0, 'Json_Spec'] = json.dumps(dict_spec)
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                df_temp.loc[0, 'Check_Url'] = check_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part=CCR-7&func=PART&vid=
[剩余数量：5] - [当前时间：13:46:22]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part=CCR-6&func=PART&vid=
[剩余数量：4] - [当前时间：13:46:22]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part=CCR-3&func=PART&vid=
[剩余数量：3] - [当前时间：13:46:22]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part=LS-312&func=PART&vid=
[剩余数量：2] - [当前时间：13:46:23]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part=DS-2282&func=PART&vid=
[剩余数量：1] - [当前时间：13:46:23]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part=NS-35&func=PART&vid=
[剩余数量：0] - [当前时间：13:46:23]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 10.https://ecatalog.smpcorp.com/V2/STD/api/part/par

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3173.90it/s]

Done ~
